# Query Classifier

## Использование

Запустите ячейки по порядку.

In [1]:
import os
import sys
import re
import joblib
import numpy as np
from pathlib import Path

# Настройка кодировки для Windows
if sys.platform == 'win32':
    try:
        if hasattr(sys.stdout, 'reconfigure'):
            sys.stdout.reconfigure(encoding='utf-8')
        if hasattr(sys.stderr, 'reconfigure'):
            sys.stderr.reconfigure(encoding='utf-8')
    except:
        pass

# Попытка импортировать sklearn
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from scipy.sparse import csr_matrix
    HAS_SKLEARN = True
except ImportError:
    HAS_SKLEARN = False
    print("[ERROR] sklearn/scipy не установлены!")
    print("Установите: pip install scikit-learn scipy")
    sys.exit(1)



In [2]:
# НАСТРОЙКИ


In [3]:
VECTORIZED_DIR = "files/vectorized"
VECTORIZER_FILE = os.path.join(VECTORIZED_DIR, "vectorizer_tfidf.pkl")

# Ключевые слова для классификации
# Включаем корни и полные слова для надежности
DELIVERY_KEYWORDS = {
    'посылк', 'посылка', 'посылки', 'посылку', 'посылкой',
    'доставк', 'доставка', 'доставки', 'доставку',
    'трек', 'трека', 'треку',
    'отправлени', 'отправление', 'отправления',
    'получ', 'получить', 'получение', 'получения',
    'забрать', 'забрал', 'забрала',
    'пункт', 'пункта', 'пункте',
    'выдач', 'выдача', 'выдачи',
    'статус', 'статуса',
    'задерж', 'задержка', 'задержки',
    'потеря', 'потерял', 'потеряла',
    'поврежд', 'повреждение', 'повреждения',
    'возврат', 'возврата',
    'отправ', 'отправка', 'отправки',
    'адрес', 'адреса',
    'курьер', 'курьера',
    'почта', 'почты',
    '5post', 'пятерочк', 'перекресток', 'магазин', 'ящик', 'локер',
    'проблем', 'проблема', 'проблемы',
    'жалоб', 'жалоба', 'жалобы',
    'ошибк', 'ошибка', 'ошибки',
    'измени', 'изменить', 'изменение',
    'отмен', 'отмена', 'отмены',
    'заказ', 'заказа', 'заказу',
    'товар', 'товара',
    'брак', 'брака',
    'ждать', 'жду',
    'придет', 'придёт', 'придет',
    'сколько', 'где', 'почему', 'когда', 'отследить',
    'постамат', 'постамата', 'постамате',
    'автомат', 'автомата', 'автомате',
    'терминал', 'терминала', 'терминале',
    'ячейк', 'ячейка', 'ячейки', 'ячейку',
    'откры', 'открыть', 'открытие',
    'пуст', 'пустая', 'пустой',
    'инвентаризац', 'инвентаризация',
    'срок хранения'
}

OPERATOR_KEYWORDS = {
    'оператор', 'менеджер', 'консультант', 'помощь', 'поддержка', 
    'связаться', 'позвонить', 'перевести', 'соединить',
    'специалист', 'человек', 'работник', 'сотрудник', 'консультация', 
    'местонахождение', 'техподдержка', 'местонахождени', 'техподдержк'
}

PROTECTED_PHRASES = {
    'не работает постамат',
    'не открылась ячейка',
    'не открывается ячейка',
    'продлить срок хранения',
    'не работает автомат',
    'пустая ячейка',
    'инвентаризация постамата',
    'постамат не работает',
    'ячейка не открылась',
    'не работает терминал',
    'автомат не работает',
    'ячейка пустая'
}

# Типы запросов с описаниями
QUERY_TYPES = {
    'delivery_and_operator': {
        'name': 'Доставка + Оператор',
        'description': 'Запрос связан с доставкой и требует связи с оператором',
        'priority': 'high'
    },
    'delivery_related': {
        'name': 'Доставка',
        'description': 'Запрос связан с доставкой посылок',
        'priority': 'high'
    },
    'operator_request': {
        'name': 'Оператор',
        'description': 'Запрос на связь с оператором',
        'priority': 'medium'
    },
    'protected_phrase': {
        'name': 'Важный запрос',
        'description': 'Известная релевантная фраза',
        'priority': 'high'
    },
    'long_phrase': {
        'name': 'Развернутый запрос',
        'description': 'Длинный запрос, требующий внимания',
        'priority': 'medium'
    },
    'too_short': {
        'name': 'Слишком короткий',
        'description': 'Запрос слишком короткий для анализа',
        'priority': 'low'
    },
    'exact_greeting': {
        'name': 'Приветствие',
        'description': 'Простое приветствие без содержания',
        'priority': 'low'
    },
    'all_non_informative': {
        'name': 'Неинформативный',
        'description': 'Запрос не содержит полезной информации',
        'priority': 'low'
    },
    'no_relevant_content': {
        'name': 'Не релевантный',
        'description': 'Запрос не относится к доставке или оператору',
        'priority': 'low'
    }
}



In [4]:
# ФУНКЦИИ


In [5]:
def basic_clean(text):
    """Базовая очистка текста"""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


def classify_query(text, vectorizer=None):
    """
    Классификация запроса
    
    Parameters:
    -----------
    text : str
        Текст запроса
    vectorizer : TfidfVectorizer, optional
        Векторизатор для дополнительного анализа
        
    Returns:
    --------
    dict : Результат классификации
    """
    if not isinstance(text, str) or not text.strip():
        return {
            'type': 'too_short',
            'confidence': 1.0,
            'is_relevant': False,
            'details': 'Пустой запрос'
        }
    
    text_clean = basic_clean(text)
    original_clean = text.strip().lower()
    
    # Проверка защищенных фраз
    if original_clean in PROTECTED_PHRASES:
        return {
            'type': 'protected_phrase',
            'confidence': 1.0,
            'is_relevant': True,
            'details': 'Известная релевантная фраза'
        }
    
    if len(text_clean) <= 2:
        return {
            'type': 'too_short',
            'confidence': 1.0,
            'is_relevant': False,
            'details': 'Запрос слишком короткий'
        }
    
    words = text_clean.split()
    
    # Проверка на неинформативные фразы
    NON_INFORMATIVE = {"спасибо", "ок", "да", "нет", "большое", "большое спасибо"}
    if original_clean in NON_INFORMATIVE:
        return {
            'type': 'exact_greeting',
            'confidence': 1.0,
            'is_relevant': False,
            'details': 'Простое приветствие'
        }
    
    if all(word in NON_INFORMATIVE for word in words):
        return {
            'type': 'all_non_informative',
            'confidence': 1.0,
            'is_relevant': False,
            'details': 'Не содержит полезной информации'
        }
    
    # Проверка ключевых слов
    # Проверяем в исходном тексте и очищенном тексте
    # Также проверяем в отдельных словах
    text_for_check = original_clean + ' ' + text_clean
    all_words = words + original_clean.split()
    
    # Явная проверка для надежности
    has_delivery = False
    for kw in DELIVERY_KEYWORDS:
        if kw in text_for_check:
            has_delivery = True
            break
        for word in all_words:
            if kw in word:
                has_delivery = True
                break
        if has_delivery:
            break
    
    has_operator = False
    for kw in OPERATOR_KEYWORDS:
        if kw in text_for_check:
            has_operator = True
            break
        for word in all_words:
            if kw in word:
                has_operator = True
                break
        if has_operator:
            break
    
    # Определение типа и уверенности
    if has_delivery and has_operator:
        return {
            'type': 'delivery_and_operator',
            'confidence': 0.95,
            'is_relevant': True,
            'details': 'Обнаружены ключевые слова: доставка + оператор'
        }
    elif has_delivery:
        return {
            'type': 'delivery_related',
            'confidence': 0.90,
            'is_relevant': True,
            'details': 'Обнаружены ключевые слова: доставка'
        }
    elif has_operator:
        return {
            'type': 'operator_request',
            'confidence': 0.85,
            'is_relevant': True,
            'details': 'Обнаружены ключевые слова: оператор'
        }
    
    # Длинные фразы могут быть релевантными
    if len(words) > 4:
        return {
            'type': 'long_phrase',
            'confidence': 0.60,
            'is_relevant': True,
            'details': 'Длинный запрос, может быть релевантным'
        }
    
    return {
        'type': 'no_relevant_content',
        'confidence': 0.80,
        'is_relevant': False,
        'details': 'Не обнаружено релевантных ключевых слов'
    }



In [6]:
def load_vectorizer(vectorizer_path):
    """Загрузка векторизатора"""
    if not os.path.exists(vectorizer_path):
        print(f"[WARN] Векторизатор не найден: {vectorizer_path}")
        print("[INFO] Будет использована классификация на основе правил")
        return None
    
    try:
        vectorizer = joblib.load(vectorizer_path)
        print(f"[OK] Векторизатор загружен из: {vectorizer_path}")
        return vectorizer
    except Exception as e:
        print(f"[WARN] Ошибка при загрузке векторизатора: {e}")
        print("[INFO] Будет использована классификация на основе правил")
        return None



In [7]:
def format_result(result):
    """Форматирование результата для вывода"""
    query_type_info = QUERY_TYPES.get(result['type'], {
        'name': result['type'],
        'description': 'Неизвестный тип',
        'priority': 'low'
    })
    
    priority_symbols = {
        'high': '[!!!]',
        'medium': '[!!]',
        'low': '[!]'
    }
    
    relevance_symbol = '[RELEVANT]' if result['is_relevant'] else '[NOT RELEVANT]'
    priority_symbol = priority_symbols.get(query_type_info['priority'], '[!]')
    
    output = []
    output.append("=" * 60)
    output.append(f"РЕЗУЛЬТАТ КЛАССИФИКАЦИИ")
    output.append("=" * 60)
    output.append(f"Тип запроса: {query_type_info['name']}")
    output.append(f"Категория: {result['type']}")
    output.append(f"Релевантность: {relevance_symbol}")
    output.append(f"Приоритет: {priority_symbol} {query_type_info['priority'].upper()}")
    output.append(f"Уверенность: {result['confidence']*100:.1f}%")
    output.append(f"Описание: {query_type_info['description']}")
    output.append(f"Детали: {result['details']}")
    output.append("=" * 60)
    
    return '\n'.join(output)



In [8]:
def interactive_mode(vectorizer=None):
    """Интерактивный режим работы"""
    print("=" * 60)
    print("ИНТЕРАКТИВНЫЙ КЛАССИФИКАТОР ЗАПРОСОВ")
    print("=" * 60)
    print("Введите запрос для классификации")
    print("Для выхода введите: 'exit', 'quit', 'выход' или 'q'")
    print("=" * 60)
    print()
    
    while True:
        try:
            # Ввод запроса
            query = input("\n[ВВОД] Введите запрос: ").strip()
            
            # Проверка на выход
            if query.lower() in ['exit', 'quit', 'выход', 'q', '']:
                print("\n[INFO] Выход из программы...")
                break
            
            if not query:
                print("[WARN] Пустой запрос. Попробуйте еще раз.")
                continue
            
            # Классификация
            print("\n[PROCESSING] Обработка запроса...")
            result = classify_query(query, vectorizer)
            
            # Вывод результата
            print("\n" + format_result(result))
            
        except KeyboardInterrupt:
            print("\n\n[INFO] Прервано пользователем. Выход...")
            break
        except Exception as e:
            print(f"\n[ERROR] Ошибка при обработке: {e}")
            import traceback
            traceback.print_exc()



In [9]:
def main(query=None, vectorizer_path=None):
    """
    Основная функция
    
    Parameters:
    -----------
    query : str, optional
        Запрос для классификации (если не указан - интерактивный режим)
    vectorizer_path : str, optional
        Путь к векторизатору (по умолчанию: VECTORIZER_FILE)
    """
    # Определяем, запущено ли в Jupyter notebook
    try:
        from IPython import get_ipython
        in_notebook = get_ipython() is not None
    except:
        in_notebook = False
    
    # Если не в notebook, используем argparse
    if not in_notebook:
        import argparse
        
        parser = argparse.ArgumentParser(
            description='Интерактивный классификатор запросов',
            formatter_class=argparse.RawDescriptionHelpFormatter,
            epilog="""
Примеры использования:
  python query_classifier.py                    # Интерактивный режим
  python query_classifier.py --query "где моя посылка"  # Один запрос
            """
        )
        
        parser.add_argument('--query', '-q',
                           type=str,
                           default=None,
                           help='Запрос для классификации (если не указан - интерактивный режим)')
        
        parser.add_argument('--vectorizer', '-v',
                           default=VECTORIZER_FILE,
                           help=f'Путь к векторизатору (по умолчанию: {VECTORIZER_FILE})')
        
        args = parser.parse_args()
        query = args.query
        vectorizer_path = args.vectorizer
    
    # Используем переданные параметры или значения по умолчанию
    if vectorizer_path is None:
        vectorizer_path = VECTORIZER_FILE
    
    # Загрузка векторизатора
    vectorizer = load_vectorizer(vectorizer_path)
    
    # Режим работы
    if query:
        # Одиночный запрос
        print(f"[INPUT] Запрос: {query}")
        result = classify_query(query, vectorizer)
        print("\n" + format_result(result))
    else:
        # Интерактивный режим
        interactive_mode(vectorizer)



In [10]:
# Для использования в notebook просто вызовите:
# main()                              # Интерактивный режим
# main(query="где моя посылка")       # Один запрос
# main(vectorizer_path="путь/к/файлу") # С другим векторизатором

# Для использования из командной строки (в .py файле):
if __name__ == "__main__":
    main()




[OK] Векторизатор загружен из: files/vectorized\vectorizer_tfidf.pkl
ИНТЕРАКТИВНЫЙ КЛАССИФИКАТОР ЗАПРОСОВ
Введите запрос для классификации
Для выхода введите: 'exit', 'quit', 'выход' или 'q'


[PROCESSING] Обработка запроса...

РЕЗУЛЬТАТ КЛАССИФИКАЦИИ
Тип запроса: Не релевантный
Категория: no_relevant_content
Релевантность: [NOT RELEVANT]
Приоритет: [!] LOW
Уверенность: 80.0%
Описание: Запрос не относится к доставке или оператору
Детали: Не обнаружено релевантных ключевых слов

[PROCESSING] Обработка запроса...

РЕЗУЛЬТАТ КЛАССИФИКАЦИИ
Тип запроса: Не релевантный
Категория: no_relevant_content
Релевантность: [NOT RELEVANT]
Приоритет: [!] LOW
Уверенность: 80.0%
Описание: Запрос не относится к доставке или оператору
Детали: Не обнаружено релевантных ключевых слов

[PROCESSING] Обработка запроса...

РЕЗУЛЬТАТ КЛАССИФИКАЦИИ
Тип запроса: Оператор
Категория: operator_request
Релевантность: [RELEVANT]
Приоритет: [!!] MEDIUM
Уверенность: 85.0%
Описание: Запрос на связь с оператором
Детали: Обн